In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

!ls

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
AChina_Predict_visual.ipynb  pycropparser.ipynb     pyviewer2.ipynb
CropAreaAugmentation.ipynb   pyield_engineer.ipynb  pyviewer.ipynb
CropArea_calc.ipynb	     pyield_export.ipynb    rename_county.ipynb
cyp			     pyield_process.ipynb   run.py
feature_Generator.ipynb      pyield_rnn2.ipynb	    task_file_generate.ipynb
Hist_analize.ipynb	     pyield_rnn3.ipynb	    TensorBoard.ipynb
images_min_max.ipynb	     pyield_rnn4.ipynb	    Untitled
parse_county.ipynb	     pyield_rnn.ipynb	    Yield_csv_clearing.ipynb


In [ ]:
import os

data_dir = '/content/gdrive/My Drive/GitHub/AChina/china/data'

home_dir = '/content/gdrive/My Drive/GitHub/AChina'
os.chdir(home_dir)
!pwd

!pip install seaborn 
!pip install geopandas

from glob import glob
import csv

import pandas as pd
import geopandas as gpd

import numpy as np

import torch

import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

from matplotlib import pyplot as plt
import matplotlib.font_manager

plt.rcParams['font.sans-serif']=['SimHei'] #Show Chinese label
plt.rcParams['axes.unicode_minus']=False

/content/gdrive/My Drive/GitHub/AChina
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
dir = os.path.join(home_dir, 'data/yields')
# dir = 'data/wheat_2002-2018.csv'
print(dir)
!pwd

files = os.path.join(dir, 'wheat_2002-2018.csv')

print(files)

shps = pd.read_csv(files)

shps

from shapely import wkt

shps['geometry'] = shps['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(shps, crs='epsg:4326')
# gdf.columns[gdf.isna().any()].tolist()
# gdf[gdf.isna().any(axis=1)]
gdf = gdf.dropna()
gdf

/content/gdrive/My Drive/GitHub/AChina/data/yields
/content/gdrive/MyDrive/GitHub/AChina
/content/gdrive/My Drive/GitHub/AChina/data/yields/wheat_2002-2018.csv


,Unnamed: 0,Value,State,County,Year,OBJECTID,BIANMA,Region_Chinese,ENAME,Shape_Leng,Shape_Area,Region,geometry,SownArea,YieldVal
1,1,1.801242,1,0,2003,1,110000.0,北京市,Beijing Shi,8.698727,1.735276,Beijing_City,"POLYGON ((116.45969 40.92497, 116.45547 40.931...",1.61,2.90
2,2,1.231707,1,0,2004,1,110000.0,北京市,Beijing Shi,8.698727,1.735276,Beijing_City,"POLYGON ((116.45969 40.92497, 116.45547 40.931...",0.82,1.01
3,3,0.625000,1,0,2005,1,110000.0,北京市,Beijing Shi,8.698727,1.735276,Beijing_City,"POLYGON ((116.45969 40.92497, 116.45547 40.931...",0.80,0.50
4,4,0.666667,1,0,2006,1,110000.0,北京市,Beijing Shi,8.698727,1.735276,Beijing_City,"POLYGON ((116.45969 40.92497, 116.45547 40.931...",0.69,0.46
5,5,0.826923,1,0,2007,1,110000.0,北京市,Beijing Shi,8.698727,1.735276,Beijing_City,"POLYGON ((116.45969 40.92497, 116.45547 40.931...",0.52,0.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522,522,0.892468,1,30,2014,31,650000.0,新疆维吾尔自治区,Xinjiangweiwu'er Zizhiqu,85.187518,175.581724,Xin_Jiang,"POLYGON ((87.02872 49.13589, 87.03313 49.13147...",82.58,73.70
523,523,0.900321,1,30,2015,31,650000.0,新疆维吾尔自治区,Xinjiangweiwu'er Zizhiqu,85.187518,175.581724,Xin_Jiang,"POLYGON ((87.02872 49.13589, 87.03313 49.13147...",74.84,67.38
524,524,0.862679,1,30,2016,31,650000.0,新疆维吾尔自治区,Xinjiangweiwu'er Zizhiqu,85.187518,175.581724,Xin_Jiang,"POLYGON ((87.02872 49.13589, 87.03313 49.13147...",83.09,71.68
525,525,0.881870,1,30,2017,31,650000.0,新疆维吾尔自治区,Xinjiangweiwu'er Zizhiqu,85.187518,175.581724,Xin_Jiang,"POLYGON ((87.02872 49.13589, 87.03313 49.13147...",74.24,65.47


In [ ]:
gdf.drop([57], inplace=True)  # 2008 year wheat for 3 county anomally

# gdf = gdf[ gdf['County'] == 3 ]

gdf

gdf['AreaWeight'] = gdf['SownArea'].max() // gdf['SownArea']    
gdf['AreaWeight'] = gdf['AreaWeight'].astype(int)

df = pd.DataFrame(gdf)
df.to_csv(os.path.join(dir, 'wheat_clear.csv'))